In [ ]:
# !pip install torch torchaudio torchvision

In [27]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 1 .Download Dataset
### 2 .Create Data Loader
### 3 .Build Model
### 4 .Train
### 5 .Save Train Model

In [28]:
# Downlaod data
def download_mnist_datasets():
  train_data = datasets.MNIST(
      root='data',
      download = True,
      train= True,
      transform = ToTensor()
  )

  validation_data = datasets.MNIST(
      root='data',
      download = True,
      train= False,
      transform = ToTensor()
  )

  return train_data, validation_data
  

In [29]:
# download dataset
train_data, _ = download_mnist_datasets()
print("Dataset downloaded")

Dataset downloaded


In [30]:
# create a dataloader (class used to wrap a dataset
                        # it's allows to load data into batches
                      #)

# for train 
BATCH_SIZE = 128
train_data_loader = DataLoader(train_data,
                               batch_size = BATCH_SIZE)



In [31]:
# Build the model
class FeedForwardNet(nn.Module):
  
  def __init__(self):
    super().__init__()
    
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential(
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self,input_data):
    
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return logits  


In [32]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(f"using device {device}")

using device cuda


In [33]:
# build the model
feed_forward_net = FeedForwardNet().to(device)

In [34]:
# train to model
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    
    # calculate loss
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    # backpropagate loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  
  print(f"Loss : {loss.item()}")


In [35]:
def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range(epochs):
    print(f"Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("---------------------------")
  
  print("Training is Done!")

In [36]:
# loss fun + optimizer
LEARNING_RATE = .001

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(
    feed_forward_net.parameters(),
    lr=LEARNING_RATE
)

In [37]:
# train the model
EPOCHS = 10
train(feed_forward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

Epoch 1
Loss : 0.2699134349822998
---------------------------
Epoch 2
Loss : 0.21806180477142334
---------------------------
Epoch 3
Loss : 0.18600553274154663
---------------------------
Epoch 4
Loss : 0.16591830551624298
---------------------------
Epoch 5
Loss : 0.1489981859922409
---------------------------
Epoch 6
Loss : 0.12755537033081055
---------------------------
Epoch 7
Loss : 0.11052163690328598
---------------------------
Epoch 8
Loss : 0.09465482085943222
---------------------------
Epoch 9
Loss : 0.07964987307786942
---------------------------
Epoch 10
Loss : 0.05756214261054993
---------------------------
Training is Done!


In [38]:
# save the model
torch.save(feed_forward_net.state_dict(), 
           "feedforwardnet.pth")
print("Model saved")

Model saved


## Loading model and making infrences

In [39]:
# load back model
# load MNIST validation dataset
# get a sample from the validation dataset for inferecee
# make a inferece

In [40]:
# load the model
feed_forword_net = FeedForwardNet()
state_dict = torch.load("/content/feedforwardnet.pth")

feed_forward_net.load_state_dict(state_dict)

<All keys matched successfully>

In [41]:
# loading MNIST dataset validate
_, validation_data = download_mnist_datasets()

In [42]:
# get a sample from the validation dataset for inferecee
input, target = validation_data[0][0],validation_data[0][1]  # first sample, target

In [43]:
class_mapping = [
                "0",
                "1",
                "2",
                "3",
                "4",
                "5",
                "6",
                "7",
                "8",
                "9"
                ]

In [45]:
def predict(model,input, target, class_mapping):
  model.eval() # evulation
  with torch.no_grad():
    predictions = model(input.to(device))
    # Tensor object  (1, 10) -> [[0.1,0.01....0.6]]
    predicted_index = predictions[0].argmax(0)
    predicted = class_mapping[predicted_index]
    expected = class_mapping[target]

    return predicted, expected

In [46]:
# make an infereces
predicted, expected = predict(feed_forward_net,input, target,class_mapping)

print(f"Predicted: {predicted} , expected: {expected}")

Predicted: 7 , expected: 7
